# Load_GeoClaw_GaugeSeries.ipynb

## Load gauge time series from netCDF file

### Modified for Newport tests

Under development for the [Cascadia CoPes Hub](https://cascadiacopeshub.org/) project, supported by NSF.

This notebook provides a function to read in a netCDF file containing gauge time series output from the GeoClaw model at a set of gauges for each event in a set of events, e.g. the 36 ground motions developed for the Cascadia CoPes Hub.


This has data for the 18 "buried rupture" events at a set of gauges in Newport, OR, for a 2-hour tsunami simulation.

The gauge locations are shown in the map at the top of [this page](https://depts.washington.edu/ptha/CopesHubTsunamis/geoclaw_runs/sites/seaside/multirun2_hyak_2024-11-11/geoclaw_plots/), or you can download the [SeasideGauges.kml](https://depts.washington.edu/ptha/CopesHubTsunamis/geoclaw_runs/sites/seaside/multirun2_hyak_2024-11-11/geoclaw_plots/SeasideGauges.kml) and open it in Google Earth.

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
import xarray
import os,sys
import pathlib
from importlib import reload
from clawpack.clawutil.util import fullpath_import

In [ ]:
try:
    CHT = os.environ['CHT']
except:
    raise Exception("*** Must first set CHT environment variable")

common_code_dir = os.path.join(CHT, 'common_code')
user_tools_dir = os.path.join(CHT, 'user_tools')

CHTtools = fullpath_import(f'{user_tools_dir}/CHTtools.py')
CHT_gaugetools = fullpath_import(f'{user_tools_dir}/CHT_gaugetools.py')

## Use this function to read in a file

This example uses the data file available at
https://depts.washington.edu/ptha/CopesHubTsunamis/data/allgauges_Seaside_random-str10.nc
To run this notebook, download this file and make sure `ncfile` includes the path to this file if it is not in the same directory as this notebook.

In [ ]:
ncfile = pathlib.Path('Newport_gauges_test_B.nc').absolute()
gauge_x, gauge_y, gauge_t, gauge_vals = CHTtools.read_allgauges_nc(ncfile)

### Examine the data:

In [ ]:
gauge_vals

To see the events included in this file you click on the disk icon next to `event` above, or print out the array of 'event' coordinates:

In [ ]:
print('Events: ',gauge_vals.coords['event'].data)

### index into numpy.ndarray

Note that  `gauge_vals.data` is an ordinary `numpy.ndarray`, but to index into this array you need to know how the dimensions or ordered, and what index corresponds to the desired gauge number, or quantity of interest, or event...

In [ ]:
gauge_vals.data.shape

### index into xarray:

It is easier to work with the `xarray.DataArray`, which provides a wrapper on top of the data to allow indexing directly by values of time, gauge number, event name, qoi name.

The `gauge_vals.sel` function allows specifying one or more dimensions.  If we specify 3 of the values as in the example below, it returns a 1-dimensional  `xarray.DataArray` where only `time` varies::

In [ ]:
mygaugeno = 1007
myevent = 'BL13D'

h = gauge_vals.sel(gaugeno=mygaugeno, qoi='h', event=myevent)
h

### Location of gauges:

`gauge_x` and `gauge_y` contain the longitude and latitude of each gauge: 

In [ ]:
gauge_vals.coords['gaugeno'].data

In [ ]:
all_gauges = gauge_vals.coords['gaugeno'].data

In [ ]:
gauge_x

In [ ]:
xg = gauge_x.sel(gaugeno=mygaugeno)
yg = gauge_y.sel(gaugeno=mygaugeno)
print(f'Gauge %i is at (%.5f, %.5f)' % (mygaugeno, xg, yg))

In [ ]:
def folium_plot_gauge(gaugenos, center=None, zoom_start=13):
    import folium
    import numpy as np

    if not isinstance(gaugenos, (list, tuple, np.ndarray)):
        # If it's not a list, wrap it in one
        gaugenos = [gaugenos]
       
    #tiles = 'OpenStreetMap'  # default tiles
    tiles = 'OpenTopoMap'   # show contours
    
    m = None

    for gaugeno in gaugenos:
 
        xg = gauge_x.sel(gaugeno=gaugeno)
        yg = gauge_y.sel(gaugeno=gaugeno)
        #print(f'Gauge %i is at (%.5f, %.5f)' % (gaugeno, xg, yg))

        if m is None:
            if center is None:
                location=(yg, xg)
            elif isinstance(center,int):
                xgc = gauge_x.sel(gaugeno=center)
                ygc = gauge_y.sel(gaugeno=center)
                location = (ygc, xgc)
            else:
                #assume center is (y,x) location
                location = center
            
            m = folium.Map(location=location, tiles=tiles, zoom_start=zoom_start)
        
        folium.Marker(
            location=[yg,xg],
            #popup = f"<b>Gauge {gaugeno}</b>\n ({xg:.6f},\n{yg:.6f})",
            popup = f"<b>Gauge {gaugeno}</b>\n {yg:.6f}N\n{-xg:.6f}W",
            tooltip="Click for info",
            icon=folium.Icon(color="red")  #, icon="cloud") # Customize the marker's appearance
        ).add_to(m) 
        
    return m

In [ ]:
folium_plot_gauge(1007)  # single gauge or list of gauges

### Make an html file with interactive map showing all gauges:

In [ ]:
map_with_all_gauges = folium_plot_gauge(all_gauges, center=(44.616,-124.015), zoom_start=13)
fname = 'Newport_gauges_folium_map.html'
map_with_all_gauges.save(fname)
print('Created ',fname)

# to display the map here, uncomment the next line:
map_with_all_gauges

### Plot the water depth `h` at one gauge, for one event:

In [ ]:
figure(figsize=(10,6))
tminutes = gauge_t / 60.
plot(tminutes, h, 'b')
grid(True)
xlabel('minutes')
ylabel('meters')
title('Water depth at Gauge %i (%.5f, %.5f)\nEvent %s' % (mygaugeno,xg,yg,myevent));

### Filtering out coarse grid values

In [ ]:
level = gauge_vals.sel(gaugeno=mygaugeno, qoi='level', event=myevent)

In [ ]:
figure(figsize=(10,3))
tminutes = gauge_t / 60.
plot(tminutes, level, 'g')
grid(True)
xlabel('minutes')
ylabel('AMR level')
title('AMR level at Gauge %i (%.5f, %.5f)\nEvent %s' % (mygaugeno,xg,yg,myevent));

In [ ]:
hfine = where(level==level.max(), h, nan)
figure(figsize=(10,6))
tminutes = gauge_t / 60.
plot(tminutes, hfine, 'b')
grid(True)
xlabel('minutes')
ylabel('meters')
title('Water depth (finest level) at Gauge %i (%.5f, %.5f)\nEvent %s' % (mygaugeno,xg,yg,myevent));

### Compute and plot momentum flux at this gauge

In [ ]:
h = gauge_vals.sel(gaugeno=mygaugeno, qoi='h', event=myevent)
u = gauge_vals.sel(gaugeno=mygaugeno, qoi='u', event=myevent)
v = gauge_vals.sel(gaugeno=mygaugeno, qoi='v', event=myevent)
mflux = h * (u**2 + v**2)

figure(figsize=(10,6))
plot(tminutes, mflux, 'b')
grid(True)
xlabel('minutes')
ylabel('m**3 / s**2')
title('Momentum flux at Gauge %i (%.5f, %.5f)\nEvent %s' % (mygaugeno,xg,yg,myevent));

### Select mulitple gauges for one event:

You can also select multiple gauges, in the case below `h` will be a 2-dimensional array indexed by `time` and `gaugeno`, but with only 4 gauges:

In [ ]:
gaugenos = range(1001,1005)
h = gauge_vals.sel(gaugeno=gaugenos, qoi='h', event=myevent)
h

In [ ]:
figure(figsize=(10,6))
for gaugeno in gaugenos: 
    plot(tminutes, h.sel(gaugeno=gaugeno), label='Gauge %i' % gaugeno)
legend()
grid(True)
xlabel('minutes')
ylabel('meters')
title('Water depth at Gauges %s\nEvent %s' % (gaugenos,myevent));

### Select a subset of events at one gauge:

In [ ]:
events = ['BR10D','BR10M']
h = gauge_vals.sel(gaugeno=mygaugeno, event=events, qoi='h')
h

In [ ]:
figure(figsize=(10,6))
for ev in events:
    plot(tminutes, h.sel(event=ev), label=ev)
legend()
grid(True)
xlabel('minutes')
ylabel('meters')
title('Water depth at Gauge %s' % mygaugeno);

### plot all the events at this gauge:

In [ ]:
events = gauge_vals.coords['event'].data  # all events
h_all = gauge_vals.sel(gaugeno=mygaugeno, event=events, qoi='h')

figure(figsize=(10,6))
for ev in events:
    plot(tminutes, h_all.sel(event=ev), label=ev)
legend()
grid(True)
xlabel('minutes')
ylabel('meters')
title('Water depth at Gauge %s' % mygaugeno);

### What's the maximum value at this gauge over all events, and which event is largest?

From the plot above it's hard to tell which event corresponds to the upper-most curve, so let's look at the maximum depth at this gauge for each event:

In [ ]:
print('The maximum depth over all time for all events is %.2f meters' % h_all.max())

We can plot the maximum depth for each event separately:

In [ ]:
hmax = h_all.max(dim='time')  # 1D array of 18 values for each event

figure(figsize=(6,7))
ievents = range(len(hmax.coords['event']))
plot(hmax.data, ievents)
yticks(ievents, hmax.coords['event'].data);
grid(True)
xlabel('meters')
title('Maximum water depth over 90 minutes at Gauge %i' % mygaugeno);

A more programatic way to determine which event has this maximum value:

In [ ]:
int(hmax.argmax(dim='event'))

`argmax` returned the integer input but we can see which event this is by indexing into the `events` array using this index:

In [ ]:
events[hmax.argmax(dim='event')]

### Plot this largest event to confirm:

In [ ]:
bigevent = events[hmax.argmax(dim='event')]
figure(figsize=(10,6))
tminutes = gauge_t / 60.
plot(tminutes, h_all.sel(event=bigevent), 'b')
grid(True)
xlabel('minutes')
ylabel('meters')
title('Water depth at Gauge %i (%.5f, %.5f)\nEvent %s' % (mygaugeno,xg,yg,bigevent));

## Ground motion during the earthquake

:::{warning}
For the latest Newport runs the ground motion is not properly captured because this gauge was not refined to the highest level until later.
:::

During the earthquake the topography `B` relative to the vertical datum (MHW here) varies with time, and hence the surface `eta = B + h` also varies. Over this short time period, `h` is roughly constant, as the entire water column above the ground moves up and down with the ground (or onshore, `h` remains 0) until the tsunami has time to form and propagate.

The data file has both `h` and `eta` and from these we can compute an plot the time-varying `B`:

In [ ]:
h = gauge_vals.sel(gaugeno=mygaugeno, qoi='h', event=myevent)
eta = gauge_vals.sel(gaugeno=mygaugeno, qoi='eta', event=myevent)
B = eta - h # topography

figure(figsize=(10,6))
plot(tminutes, B, 'g')
#plot(tminutes, eta, 'b')
grid(True)
xlabel('minutes')
ylabel('meters')
xlim(0,20)
title('Ground motion at Gauge %i (%.5f, %.5f)\nEvent %s' % (mygaugeno,xg,yg,myevent));